In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [2]:
import os
os.chdir("/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/Code")

In [ ]:
from models_methods.data.preprocessing import Autoaugment_preprocess
from models_methods.utility.bypass_bn import enable_running_stats, disable_running_stats
from models_methods.utility.early_stopping import EarlyStopping
from train import main 

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# get data

# note: autougment doesn't seem to work very well 
transform = Autoaugment_preprocess(channels=3, resize_dim=(36,36), crop_dim=(28,28)) 


root = "/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/data" # change this to your data directory

trainset = torchvision.datasets.CIFAR10(root=root, train=True,
                                      download=True, transform=transform.transform)

trainset, valset = torch.utils.data.random_split(trainset, [45000, 5000])

testset = torchvision.datasets.CIFAR10(root=root, train=False,
                                      download=True, transform=transform.transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=64,
                                        shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

data_loaders = {
    "train_loader": trainloader,
    "val_loader": valloader,
    "test_loader": testloader
}          

In [6]:
# create simple model 

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
main(
    model = SimpleCNN(),
    optimizer = torch.optim.SGD,
    loss_fn=smooth_crossentropy,
    train_step=train_step,
    test_step=test_step,
    data_loaders=data_loaders,
    use_SAM=True, 
    device=device,
    exp_name="TEST_1",
    epochs=5,
    use_early_stopping=True,
    delta=1e-3,
    verbose_steps=True,
    log_interval=100,
    smoothing=0.1
    )

NameError: name 'SAM' is not defined

In [ ]:
# visualize with tensorboard
%reload_ext tensorboard
%tensorboard --logdir=/home/peppe/01_Study/01_University/Semester/2/Intro_to_ML/Project/Code/experiments/sam_test_2 # experiment path